<div style="background-color: #1f77b4; color: #fff; border-radius: 10px; padding: 10px;">
    <H2>Doordash Efficiency Project</H2>
</div>






<div style="background-color: #b3d7ff; color: #000; border-radius: 10px; padding: 10px;">
<h3
    >Based on information provided from Doordash, I need to determine strategy on most profitable insights, based on earning I've got for the past 10 months.


<div style="background-color: #b3d7ff; color: #000; border-radius: 10px; padding: 10px;">
<h3
    > STEP 1. Let`s check what data we have so far and import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dash = pd.read_csv('dasher_delivery_information.csv')
dash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ORDER_CREATED_TIME    488 non-null    object 
 1   ACTUAL_PICKUP_TIME    488 non-null    object 
 2   ACTUAL_DELIVERY_TIME  488 non-null    object 
 3   STORE_NAME            488 non-null    object 
 4   TOTAL_ITEM_COUNT      488 non-null    int64  
 5   SUBTOTAL_IN_CENTS     488 non-null    float64
 6   ORDER_STATUS          488 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 26.8+ KB


In [3]:
dash.columns = dash.columns.str.lower()
dash['order_created_time'] = pd.to_datetime(dash['order_created_time']).dt.strftime('%Y-%m-%d %H:%M')
dash['actual_pickup_time'] = pd.to_datetime(dash['actual_pickup_time']).dt.strftime('%Y-%m-%d %H:%M')
dash['actual_delivery_time'] = pd.to_datetime(dash['actual_delivery_time']).dt.strftime('%Y-%m-%d %H:%M')

dash.head()

,order_created_time,actual_pickup_time,actual_delivery_time,store_name,total_item_count,subtotal_in_cents,order_status
0,2022-10-05 20:47,2022-10-05 20:53,2022-10-05 21:51,Jersey Mike's Subs (3215),3,1660.0,Order Delivered
1,2022-10-05 22:33,2022-10-05 22:49,2022-10-05 22:56,Marco's Pizza (8403),3,2187.0,Order Delivered
2,2022-10-05 22:39,2022-10-05 23:25,2022-10-05 23:49,Cold Stone Creamery (21465),1,1365.0,Order Delivered
3,2022-10-05 22:45,2022-10-05 23:08,2022-10-05 23:18,Glory Days Grill (Apex),1,1455.0,Order Delivered
4,2022-10-05 23:44,2022-10-06 00:13,2022-10-06 00:20,Wendy's (9396),2,2435.0,Order Delivered


In [4]:
dash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   order_created_time    488 non-null    object 
 1   actual_pickup_time    488 non-null    object 
 2   actual_delivery_time  488 non-null    object 
 3   store_name            488 non-null    object 
 4   total_item_count      488 non-null    int64  
 5   subtotal_in_cents     488 non-null    float64
 6   order_status          488 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 26.8+ KB


# Data:
1. **order_created_time**: Time when I accepted the order. The difference between this time and pickup time would help me determine how long it takes me to actually *start* delivering - the thing I get paid for.

2. **time I've picked order and headed to the customer**: This is a check mark for me to understand how much time I usually spend actually delivering the order.

3. **time I've delivered**: The time when I successfully delivered the order.

4. **store_name**: We would determine the most popular store in the list and delivering more gross store.

5. **subtotal_in_cents**: We would convert it to dollars. This column shows the price of the order I've delivered. Presumably, the more the cost of items, the bigger the tips. Also, I want to check whether the biggest orders take longer to deliver.

6. **order_status**: Need to check whether there are not delivered orders.


Converting subtotal_in_cents to dollar amount

In [5]:
dash['order_price'] = (dash['subtotal_in_cents']/100).round(2)
dash.sort_values( by = 'order_price', ascending = False).head(7)


,order_created_time,actual_pickup_time,actual_delivery_time,store_name,total_item_count,subtotal_in_cents,order_status,order_price
148,2022-10-30 23:01,2022-10-31 00:03,2022-10-31 00:20,Outback Steakhouse - (3443 OBS CARY NC),5,8803.0,Order Delivered,88.03
196,2022-11-16 22:45,2022-11-16 23:14,2022-11-16 23:42,Sushi Iwa (Creekside Landing Dr),6,8625.0,Order Delivered,86.25
346,2023-06-13 22:01,2023-06-13 22:36,2023-06-13 22:55,Sushi at the Park,4,7935.0,Order Delivered,79.35
361,2023-06-20 00:01,2023-06-20 00:18,2023-06-20 00:28,Sushi Iwa (Creekside Landing Dr),7,7080.0,Order Delivered,70.80
233,2022-11-27 01:33,2022-11-27 02:14,2022-11-27 02:28,Hyderabad House Biryani Place (Morrisville),4,6896.0,Order Delivered,68.96
383,2023-06-28 18:00,2023-06-28 18:53,2023-06-28 19:27,The Cheesecake Factory (Durham - 0102),7,6885.0,Order Delivered,68.85
329,2023-01-28 01:21,2023-01-28 01:47,2023-01-28 02:02,Sushi Iwa (Creekside Landing Dr),6,6535.0,Order Delivered,65.35


Calculating times: 
1. time_to_get
2. time_to_deliver

In [9]:
dash['time_to_get'] = (pd.to_datetime(dash['actual_pickup_time']) - pd.to_datetime(dash['order_created_time'])).astype('timedelta64[m]') #.dt.total_seconds()/60).round(1)
dash['time_to_deliver'] = (pd.to_datetime(dash['actual_delivery_time']) - pd.to_datetime(dash['actual_pickup_time'])).astype('timedelta64[m]')
dash.head()

,order_created_time,actual_pickup_time,actual_delivery_time,store_name,total_item_count,subtotal_in_cents,order_status,order_price,time_to_get,time_to_deliver
0,2022-10-05 20:47,2022-10-05 20:53,2022-10-05 21:51,Jersey Mike's Subs (3215),3,1660.0,Order Delivered,16.60,6.0,58.0
1,2022-10-05 22:33,2022-10-05 22:49,2022-10-05 22:56,Marco's Pizza (8403),3,2187.0,Order Delivered,21.87,16.0,7.0
2,2022-10-05 22:39,2022-10-05 23:25,2022-10-05 23:49,Cold Stone Creamery (21465),1,1365.0,Order Delivered,13.65,46.0,24.0
3,2022-10-05 22:45,2022-10-05 23:08,2022-10-05 23:18,Glory Days Grill (Apex),1,1455.0,Order Delivered,14.55,23.0,10.0
4,2022-10-05 23:44,2022-10-06 00:13,2022-10-06 00:20,Wendy's (9396),2,2435.0,Order Delivered,24.35,29.0,7.0


Let's check if there are some excessive numbers that interfere with the average.


In [10]:
dash.time_to_get.describe()

count     488.000000
mean       28.903689
std        80.820059
min         2.000000
25%        15.000000
50%        20.500000
75%        28.000000
max      1664.000000
Name: time_to_get, dtype: float64

In [24]:
dash[
    (dash['order_created_time'] >= '2023-07-18 00:00:00') &
    (dash['order_created_time'] <= '2023-07-18 23:59:59')
]

#dash.time_to_get.describe()

,order_created_time,actual_pickup_time,actual_delivery_time,store_name,total_item_count,subtotal_in_cents,order_status,order_price,time_to_get,time_to_deliver
436,2023-07-18 19:20,2023-07-18 21:07,2023-07-18 21:18,JJ China Restaurant (Morrisville),4,1374.0,Order Delivered,13.74,107.0,11.0
437,2023-07-18 21:05,2023-07-18 22:00,2023-07-18 22:01,Lacucina italiana(Halal Food),2,1924.0,Order Delivered,19.24,55.0,1.0
438,2023-07-18 22:06,2023-07-18 22:17,2023-07-18 22:25,Dairy Queen (10398),2,1797.0,Order Delivered,17.97,11.0,8.0
439,2023-07-18 22:22,2023-07-18 22:49,2023-07-18 23:01,Chick-fil-A (00833),1,1302.0,Order Delivered,13.02,27.0,12.0
440,2023-07-18 22:25,2023-07-18 22:38,2023-07-18 23:00,Walgreens (9757),1,829.0,Order Delivered,8.29,13.0,22.0


In [ ]:
plt.figure(figsize=(14, 6)) 
plt.hist(dash['time_to_get'], color='green', edgecolor='black', alpha=0.7)
plt.xlabel('Rooms', fontsize = 12)
plt.ylabel('Listing amount', fontsize = 12)
plt.title('Rooms per apartment', fontsize = 15)
plt.show()